# **PREDICTING RELEVANT ACTORS FOR A GENRE**

<br>

<br>

**1.1. LIBRARY IMPORTING**

In [68]:
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer


**1.2. LOAD DATASETS**

In [69]:
# Load the datasets
movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
credits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

# Display basic information about the datasets
print("Movies Dataset Columns:")
print(movies.columns)

print("\nCredits Dataset Columns:")
print(credits.columns)


Movies Dataset Columns:
Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

Credits Dataset Columns:
Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')


**1.3.  MERGE THE DATASETS**

In [70]:
# Merge the datasets on the 'title' column
merged_data = pd.merge(movies, credits, on="title")

# Retain only relevant columns
columns_to_keep = ['title', 'genres', 'cast']
merged_data = merged_data[columns_to_keep]

# Display merged dataset
print("Merged Dataset:")
print(merged_data.head())

Merged Dataset:
                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                              genres  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                                cast  
0  [{"cast_id": 242, "character": "Jake Sully", "...  
1  [{"cast_id": 4, "character": "Captain Jack Spa...  
2  [{"cast_id": 1, "character": "James Bond", "cr...  
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...  
4  [{"cast_id": 5, "character": "John Carter", "c...  


- Merge the datasets to unify the `genres` and `cast` columns for each movie.
- Keep only the columns necessary for analyzing genres and cast, which are critical for the model.

**1.4. PROCESS GENRES**

In [71]:
# Extract genre names from the JSON-like column
def extract_genres(genres_column):
    try:
        return [genre['name'] for genre in ast.literal_eval(genres_column)]
    except (ValueError, SyntaxError):
        return []

merged_data['genres'] = merged_data['genres'].apply(extract_genres)

# Display processed genres
print("Processed Genres:")
print(merged_data[['title', 'genres']].head())

Processed Genres:
                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                          genres  
0  [Action, Adventure, Fantasy, Science Fiction]  
1                   [Adventure, Fantasy, Action]  
2                     [Action, Adventure, Crime]  
3               [Action, Crime, Drama, Thriller]  
4           [Action, Adventure, Science Fiction]  


Explanation:

- Parse the `genres` column to extract a list of genre names for each movie.
- This ensures the data is ready for genre-based analysis.

**1.5. PROCESS CAST**

In [72]:
# Extract top 3 actor names from the JSON-like column
def extract_top_actors(cast_column):
    try:
        return [actor['name'] for actor in ast.literal_eval(cast_column)[:3]]  # Top 3 actors
    except (ValueError, SyntaxError):
        return []

merged_data['cast'] = merged_data['cast'].apply(extract_top_actors)

# Display processed cast
print("Processed Cast:")
print(merged_data[['title', 'cast']].head())

Processed Cast:
                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                               cast  
0  [Sam Worthington, Zoe Saldana, Sigourney Weaver]  
1     [Johnny Depp, Orlando Bloom, Keira Knightley]  
2      [Daniel Craig, Christoph Waltz, Léa Seydoux]  
3      [Christian Bale, Michael Caine, Gary Oldman]  
4    [Taylor Kitsch, Lynn Collins, Samantha Morton]  


- Parse the `cast` column to extract the names of the top 3 actors for each movie.
- These actors are the candidates for determining relevance by genre

**FINAL DATASET STRUCTURE**
After processing, the dataset will have:

- `title`: Movie title (for reference).
- `genres`: List of genres for the movie.
- `cast`: Top 3 actors' names.

This ensures the data is now prepared for further steps to analyze the relationship between genres and relevant actors.

<br>

# **STEP 2: FEATURE ENGINEERING**

**2.1. ENCODE `genres`**

In [73]:
mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(merged_data['genres']), columns=mlb.classes_)

# Add encoded genres back to the dataset
merged_data = pd.concat([merged_data, genres_encoded], axis=1)

# Drop the original 'genres' column
merged_data.drop('genres', axis=1, inplace=True)

# Display the dataset with encoded genres
print("Dataset with Encoded Genres:")
print(merged_data.head())


Dataset with Encoded Genres:
                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                               cast  Action  Adventure  \
0  [Sam Worthington, Zoe Saldana, Sigourney Weaver]       1          1   
1     [Johnny Depp, Orlando Bloom, Keira Knightley]       1          1   
2      [Daniel Craig, Christoph Waltz, Léa Seydoux]       1          1   
3      [Christian Bale, Michael Caine, Gary Oldman]       1          0   
4    [Taylor Kitsch, Lynn Collins, Samantha Morton]       1          1   

   Animation  Comedy  Crime  Documentary  Drama  Family  ...  History  Horror  \
0          0       0      0            0      0       0  ...        0       0   
1          0       0      0            0      0       0  ...        0       0   


- Use `MultiLabelBinarizer` to one-hot encode the `genres` column.
- Each genre becomes a separate binary feature (1 = present, 0 = absent).

**2.2 CREATE ACTOR FEATURES**


In [74]:
# Flatten the cast list into individual actor columns
for i in range(3):
    merged_data[f'actor_{i+1}'] = merged_data['cast'].apply(lambda x: x[i] if len(x) > i else None)

# Drop the original 'cast' column
merged_data.drop('cast', axis=1, inplace=True)



In [75]:
# Display dataset with actor columns
print("Dataset with Actor Features:")
merged_data.head()

Dataset with Actor Features:


,title,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,actor_1,actor_2,actor_3
0,Avatar,1,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,Sam Worthington,Zoe Saldana,Sigourney Weaver
1,Pirates of the Caribbean: At World's End,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,Johnny Depp,Orlando Bloom,Keira Knightley
2,Spectre,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,Daniel Craig,Christoph Waltz,Léa Seydoux
3,The Dark Knight Rises,1,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,Christian Bale,Michael Caine,Gary Oldman
4,John Carter,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,Taylor Kitsch,Lynn Collins,Samantha Morton


- Extract the top 3 actors from the `cast` column into separate columns (`actor_1`, `actor_2`, `actor_3`).
- Simplifies analysis by making actor data explicit.

**2.3 ASSIGN GENRE-SPECIFIC ACTOR RELEVANCE**

In [76]:
# Create a helper DataFrame to track actor relevance by genre
actor_genre_relevance = merged_data.melt(id_vars=mlb.classes_, value_vars=['actor_1', 'actor_2', 'actor_3'], 
                                         var_name='actor_position', value_name='actor')

# Drop rows where 'actor' is NaN
actor_genre_relevance.dropna(subset=['actor'], inplace=True)



In [77]:
# Display the actor-genre mapping
print("ACTOR-GENRE RELEVANCE DataFrame:")
actor_genre_relevance.head()

ACTOR-GENRE RELEVANCE DataFrame:


,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,...,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,actor_position,actor
0,1,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,actor_1,Sam Worthington
1,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,actor_1,Johnny Depp
2,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,actor_1,Daniel Craig
3,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,actor_1,Christian Bale
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,actor_1,Taylor Kitsch


- Create a mapping between genres and actors.
- Use the `melt` function to transform actor columns into rows, associating each actor with their respective genres.


**2.4. GENERATE TRAINIG DATASET**

In [78]:
# Aggregate actor appearances across genres
actor_genre_count = actor_genre_relevance.groupby(['actor'] + mlb.classes_).size().reset_index(name='count')

# Filter out less relevant actors (e.g., threshold of appearances)
threshold = 2
actor_genre_count = actor_genre_count[actor_genre_count['count'] >= threshold]

# Display the final training dataset
print("Training Dataset:")
print(actor_genre_count.head())


ValueError: Grouper and axis must be same length